# Data preperation
In this notebook the data is prepared for the clustering algortihm

## Input:
- Joined layer shapefile in which all BAG elements of the inner city and their state of protection are given: 'bag_protected_joined.shp' 
- The shapefile with the buildings in the inner city: 'arch_buildings_centrum.shp' 
- The datafiles in the map Residential_demand containing the demand per buildingtype per hour per m2 
- The shapefile with the buildings containing the potential waste heat in the innercity: arch_non_residential_WH.gpkg 
- The datafiles in the map ElectricityDemand containing electricity demand per hour per non-residential building function

## Output:
- Dataframe containing the hourly data of all residential and non-residential buildings and their hourly demand or supply
- Dataframe containing the data of all residential and non-residential buildings and their demand or supply over different time periods (whole year, coldest day and coldest month)

In [1]:
# Load in the necessary packages
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import warnings

### 1. connecting the adresses to the retrofitted demand

In [2]:
# Import the BAG objects, this is the data where we want to couple the retrofitted demand to
bag = gpd.read_file('data/bag_protected_joined/bag_protected_joined.shp')

In [3]:
# Check columns of this shapefile 
bag.columns

Index(['gid', 'identifica', 'oppervlakt', 'status', 'gebruiksdo', 'openbare_r',
       'huisnummer', 'huisletter', 'toevoeging', 'postcode', 'woonplaats',
       'bouwjaar', 'pandidenti', 'pandstatus', 'rdf_seeals', 'Objectnumm',
       'Orde', 'SELECTIE_O', 'Gebied', 'SELECTIE_G', 'Architectn', 'Architect',
       'Bouwjaar_2', 'Stadsdeel', 'Brug', '_errors', 'layer', 'path', 'date_f',
       'Joined lay', 'geometry'],
      dtype='object')

In [4]:
# Short data analysis
print('Value counts of bouwjaar:\n',bag['bouwjaar'].value_counts())
print('\n')
print('Description of building functions:\n',bag['gebruiksdo'].describe())

Value counts of bouwjaar:
 1005    22636
1905     5065
1900     1312
1990     1219
1989     1121
        ...  
1766        1
1772        1
1714        1
1793        1
1635        1
Name: bouwjaar, Length: 338, dtype: int64


Description of building functions:
 count           66126
unique             51
top       woonfunctie
freq            54870
Name: gebruiksdo, dtype: object


In [5]:
# Only keep residential buildings
bag_w = bag[bag['gebruiksdo'] == 'woonfunctie']

# Select relevant columns
bag_w = bag_w[['pandidenti', 'oppervlakt', 'bouwjaar','Orde','geometry','postcode','openbare_r']]

# Preview of dataframe
bag_w.head()

,pandidenti,oppervlakt,bouwjaar,Orde,geometry,postcode,openbare_r
0,0363100012179842,95,1005,2,POINT (121231.000 488105.000),1013ED,Herenmarkt
1,0363100012179842,57,1005,2,POINT (121231.000 488105.000),1013ED,Herenmarkt
2,0363100012179842,114,1005,2,POINT (121231.000 488105.000),1013ED,Herenmarkt
4,0363100012168211,110,1875,2,POINT (121245.000 487930.000),1015BB,Herengracht
5,0363100012168211,87,1875,2,POINT (121245.000 487930.000),1015BB,Herengracht


In [6]:
# Import the building data, this is needed to couple the building type to the BAG data
arch = gpd.read_file('data/arch_buildings_centrum/arch_buildings_centrum.shp')

In [7]:
arch.head()

,identifica,rdf_seeals,bouwjaar,status,gebruiksdo,oppervlakt,oppervla_1,aantal_ver,fuuid,woningtype,wwr,geometry
0,0363100012179956,http://bag.basisregistraties.overheid.nl/bag/i...,1776.0,Pand in gebruik,"kantoorfunctie,woonfunctie",32.0,95.0,5.0,pand.1b5222d9-b6af-4f38-ab66-6c7abcd8bc8b,appartement,NaN,"POLYGON ((121499.111 487058.546, 121497.284 48..."
1,0363100012165291,http://bag.basisregistraties.overheid.nl/bag/i...,1663.0,Pand in gebruik,"kantoorfunctie,woonfunctie",100.0,200.0,2.0,pand.e012b44c-8819-4700-80b9-202366780d72,appartement,0.42,"POLYGON ((121499.099 487054.510, 121517.428 48..."
2,0363100012165292,http://bag.basisregistraties.overheid.nl/bag/i...,1742.0,Pand in gebruik,woonfunctie,55.0,87.0,4.0,pand.bf26870e-9633-4449-a02b-3fd109010fee,appartement,NaN,"POLYGON ((121515.430 487053.906, 121519.341 48..."
3,0363100012165293,http://bag.basisregistraties.overheid.nl/bag/i...,1994.0,Pand in gebruik,"bijeenkomstfunctie,logiesfunctie,overige gebru...",1.0,28249.0,15.0,pand.a264f424-1bbd-4e25-9003-98b1d60ab047,None,NaN,"POLYGON ((121513.315 487313.928, 121507.630 48..."
4,0363100012165369,http://bag.basisregistraties.overheid.nl/bag/i...,1754.0,Pand in gebruik,woonfunctie,41.0,70.0,22.0,pand.16b37d07-7081-4310-a719-0aad00b96484,appartement,NaN,"POLYGON ((121541.713 487297.493, 121539.536 48..."


In [8]:
# We will couple the buildings to the BAG data based on pand identity
arch = arch[['identifica', 'woningtype']]
arch.rename(columns = {'identifica':'pandidenti'}, inplace = True)
arch['woningtype'].unique()

array(['appartement', None, 'tussenwoning/geschakeld', 'hoekwoning',
       'twee-onder-een-kap', 'vrijstaande woning'], dtype=object)

In [9]:
# How many NaN values?
arch['woningtype'].isna().sum()

3268

In [10]:
# Now couple arch and bag based on the pandidentity
df = pd.merge(bag_w, arch, on='pandidenti')
df.head()

,pandidenti,oppervlakt,bouwjaar,Orde,geometry,postcode,openbare_r,woningtype
0,0363100012179842,95,1005,2,POINT (121231.000 488105.000),1013ED,Herenmarkt,appartement
1,0363100012179842,57,1005,2,POINT (121231.000 488105.000),1013ED,Herenmarkt,appartement
2,0363100012179842,114,1005,2,POINT (121231.000 488105.000),1013ED,Herenmarkt,appartement
3,0363100012168211,110,1875,2,POINT (121245.000 487930.000),1015BB,Herengracht,appartement
4,0363100012168211,87,1875,2,POINT (121245.000 487930.000),1015BB,Herengracht,appartement


In [11]:
df.shape

(54787, 8)

In [12]:
df['x'] = df['geometry'].x
df['y'] = df['geometry'].y
df['point'] = df['x'].astype(str) + ', ' + df['y'].astype(str)

df is the dataframe ready to couple the retrofitted data to

### 2. Couple the retrofitted data by category to the buildings in the inner city

In [13]:
# Now couple this dataframe to the retrofitted data set
# We have to split the dataframe into different years
# For the data before 1945 we also have to check if the building is protected, based on the column 'orde'

df45 = df.loc[df['bouwjaar'] <= '1945']
df45_np = df45[df45['Orde'].isna()]
df45_p = df45[df45['Orde'].notna()]
df46_74 = df.loc[(df['bouwjaar'] <= '1974') & (df['bouwjaar'] >= '1946')]
df75_91 = df.loc[(df['bouwjaar'] <= '1991') & (df['bouwjaar'] >= '1975')]
df92 = df.loc[df['bouwjaar'] >= '1992']

In [14]:
# Now split the dataframe in even more sets of dataframe based on their woningtype

df45_np_a = df45_np.loc[df45_np['woningtype'] == 'appartement']
df45_np_t = df45_np.loc[df45_np['woningtype'] == 'tussenwoning/geschakeld']
df45_np_sd = df45_np.loc[df45_np['woningtype'] == 'hoekwoning']
df45_p_a = df45_p.loc[df45_p['woningtype'] == 'appartement']
df45_p_t = df45_p.loc[df45_p['woningtype'] == 'tussenwoning/geschakeld']
df45_p_sd = df45_p.loc[df45_p['woningtype'] == 'hoekwoning']

In [15]:
df46_74_a = df46_74.loc[df46_74['woningtype'] == 'appartement']
df46_74_t = df46_74.loc[df46_74['woningtype'] == 'tussenwoning/geschakeld']
df46_74_sd = df46_74.loc[df46_74['woningtype'] == 'hoekwoning']

In [16]:
df75_91_a = df75_91.loc[df75_91['woningtype'] == 'appartement']
df75_91_t = df75_91.loc[df75_91['woningtype'] == 'tussenwoning/geschakeld']
df75_91_sd = df75_91.loc[df75_91['woningtype'] == 'hoekwoning']

In [17]:
df92_a = df92.loc[df92['woningtype'] == 'appartement']
df92_t = df92.loc[df92['woningtype'] == 'tussenwoning/geschakeld']
df92_sd = df92.loc[df92['woningtype'] == 'hoekwoning']

In [18]:
# For each dataframe couple the heating, hot water and cooling demand per m2 and multiply this with the surface of the building
# heating, hot water and cooling demand per m2 will be summed, where cooling is seen as supply

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

dem45_p_a = pd.read_excel('data/Residential_demand/Apartment/1_apartment_before1945/LT_appart_b_1946_protected.xlsx')

df45_p_a = df45_p_a[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']]
df45_p_a = df45_p_a.reset_index(drop=True)

dem45_p_a['total'] = - dem45_p_a['heating_kWh/m2'] - dem45_p_a['hot_water_kWh/m2'] + dem45_p_a['cooling_kWh/m2']
dem45_p_a = pd.DataFrame([dem45_p_a['total']])
dem45_p_a = pd.concat([dem45_p_a]*len(df45_p_a), ignore_index=True)

df45_p_a_t = pd.concat([df45_p_a[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']].T, dem45_p_a.T]).T

df45_p_a_t.update(df45_p_a_t.iloc[:, 7:8759+8].mul(df45_p_a_t.oppervlakt, 0))

In [19]:
dem45_p_t = pd.read_excel('data/Residential_demand/Terraced/row_b_1945/LT_row_b_1945_protected_fin.xlsx')

df45_p_t = df45_p_t[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']]
df45_p_t = df45_p_t.reset_index(drop=True)

dem45_p_t['total'] = - dem45_p_t['heating_kWh/m2'] - dem45_p_t['hot_water_kWh/m2'] + dem45_p_t['cooling_kWh/m2']
dem45_p_t = pd.DataFrame([dem45_p_t['total']])
dem45_p_t = pd.concat([dem45_p_t]*len(df45_p_t), ignore_index=True)

df45_p_t_t = pd.concat([df45_p_t[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']].T, dem45_p_t.T]).T

df45_p_t_t.update(df45_p_t_t.iloc[:, 7:8759+8].mul(df45_p_t_t.oppervlakt, 0))
df45_p_t_t = df45_p_t_t.iloc[:, 0:8759+8]

In [20]:
dem45_p_sd = pd.read_excel('data/Residential_demand/Semi-detached/corner_b_1945/LT_corner_b_1945_protected_fin.xlsx')

df45_p_sd = df45_p_sd[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']]
df45_p_sd = df45_p_sd.reset_index(drop=True)

dem45_p_sd['total'] = - dem45_p_sd['heating_kWh/m2'] - dem45_p_sd['hot_water_kWh/m2'] + dem45_p_sd['cooling_kWh/m2']
dem45_p_sd = pd.DataFrame([dem45_p_sd['total']])
dem45_p_sd = pd.concat([dem45_p_sd]*len(df45_p_sd), ignore_index=True)

df45_p_sd_t = pd.concat([df45_p_sd[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']].T, dem45_p_sd.T]).T

df45_p_sd_t.update(df45_p_sd_t.iloc[:, 7:8759+8].mul(df45_p_sd_t.oppervlakt, 0))
df45_p_sd_t = df45_p_sd_t.iloc[:, 0:8759+8]

In [21]:
dem45_np_a = pd.read_excel('data/Residential_demand/Apartment/1_apartment_before1945/LT_appart_b_1946_not_protected.xlsx')

df45_np_a = df45_np_a[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']]
df45_np_a = df45_np_a.reset_index(drop=True)

dem45_np_a['total'] = - dem45_np_a['heating_kWh/m2'] - dem45_np_a['hot_water_kWh/m2'] + dem45_np_a['cooling_kWh/m2']
dem45_np_a = pd.DataFrame([dem45_np_a['total']])
dem45_np_a = pd.concat([dem45_np_a]*len(df45_np_a), ignore_index=True)

df45_np_a_t = pd.concat([df45_np_a[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']].T, dem45_np_a.T]).T

df45_np_a_t.update(df45_np_a_t.iloc[:, 7:8759+8].mul(df45_np_a_t.oppervlakt, 0))

In [22]:
dem45_np_t = pd.read_excel('data/Residential_demand/Terraced/row_b_1945/LT_row_b_1945_n_protected_fin.xlsx')

df45_np_t = df45_np_t[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']]
df45_np_t = df45_np_t.reset_index(drop=True)

dem45_np_t['total'] = - dem45_np_t['heating_kWh/m2'] - dem45_np_t['hot_water_kWh/m2'] + dem45_np_t['cooling_kWh/m2']
dem45_np_t = pd.DataFrame([dem45_np_t['total']])
dem45_np_t = pd.concat([dem45_np_t]*len(df45_np_t), ignore_index=True)

df45_np_t_t = pd.concat([df45_np_t[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']].T, dem45_np_t.T]).T

df45_np_t_t.update(df45_np_t_t.iloc[:, 7:8759+8].mul(df45_np_t_t.oppervlakt, 0))
df45_np_t_t = df45_np_t_t.iloc[:, 0:8759+8]

In [23]:
dem45_np_sd = pd.read_excel('data/Residential_demand/Semi-detached/corner_b_1945/LT_corner_b_1945_n_protected_fin.xlsx')

df45_np_sd = df45_np_sd[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']]
df45_np_sd = df45_np_sd.reset_index(drop=True)

dem45_np_sd['total'] = - dem45_np_sd['heating_kWh/m2'] - dem45_np_sd['hot_water_kWh/m2'] + dem45_np_sd['cooling_kWh/m2']
dem45_np_sd = pd.DataFrame([dem45_np_sd['total']])
dem45_np_sd = pd.concat([dem45_np_sd]*len(df45_np_sd), ignore_index=True)

df45_np_sd_t = pd.concat([df45_np_sd[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']].T, dem45_np_sd.T]).T

df45_np_sd_t.update(df45_np_sd_t.iloc[:, 7:8759+8].mul(df45_np_sd_t.oppervlakt, 0))
df45_np_sd_t = df45_np_sd_t.iloc[:, 0:8759+8]

In [24]:
dem46_74_a = pd.read_excel('data/Residential_demand/Apartment/2_apartment_1945_1974/LT_appart_1946_1974_cavityw.xlsx')

df46_74_a = df46_74_a[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']]
df46_74_a = df46_74_a.reset_index(drop=True)

dem46_74_a['total'] = - dem46_74_a['heating_kWh/m2'] - dem46_74_a['hot_water_kWh/m2'] + dem46_74_a['cooling_kWh/m2']
dem46_74_a = pd.DataFrame([dem46_74_a['total']])
dem46_74_a = pd.concat([dem46_74_a]*len(df46_74_a), ignore_index=True)

df46_74_a_t = pd.concat([df46_74_a[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']].T, dem46_74_a.T]).T

df46_74_a_t.update(df46_74_a_t.iloc[:, 7:8759+8].mul(df46_74_a_t.oppervlakt, 0))

In [25]:
dem46_74_t = pd.read_excel('data/Residential_demand/Terraced/row_1945_1974/LT_row_1946_1974_cavity_fin.xlsx')

df46_74_t = df46_74_t[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']]
df46_74_t = df46_74_t.reset_index(drop=True)

dem46_74_t['total'] = - dem46_74_t['heating_kWh/m2'] - dem46_74_t['hot_water_kWh/m2'] + dem46_74_t['cooling_kWh/m2']
dem46_74_t = pd.DataFrame([dem46_74_t['total']])
dem46_74_t = pd.concat([dem46_74_t]*len(df46_74_t), ignore_index=True)

df46_74_t_t = pd.concat([df46_74_t[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']].T, dem46_74_t.T]).T

df46_74_t_t.update(df46_74_t_t.iloc[:, 7:8759+8].mul(df46_74_t_t.oppervlakt, 0))
df46_74_t_t = df46_74_t_t.iloc[:, 0:8759+8]

In [26]:
dem75_91_a = pd.read_excel('data/Residential_demand/Apartment/3_apartment_1975_1991/LT_appart_1975_1991_cavityw.xlsx')

df75_91_a = df75_91_a[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']]
df75_91_a = df75_91_a.reset_index(drop=True)

dem75_91_a['total'] = - dem75_91_a['heating_kWh/m2'] - dem75_91_a['hot_water_kWh/m2'] + dem75_91_a['cooling_kWh/m2']
dem75_91_a = pd.DataFrame([dem75_91_a['total']])
dem75_91_a = pd.concat([dem75_91_a]*len(df75_91_a), ignore_index=True)

df75_91_a_t = pd.concat([df75_91_a[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']].T, dem75_91_a.T]).T

df75_91_a_t.update(df75_91_a_t.iloc[:, 7:8759+8].mul(df75_91_a_t.oppervlakt, 0))

In [27]:
dem75_91_t = pd.read_excel('data/Residential_demand/Terraced/row_1974_1992/LT_row_1975_1992_cavity_fin.xlsx')

df75_91_t = df75_91_t[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']]
df75_91_t = df75_91_t.reset_index(drop=True)

dem75_91_t['total'] = - dem75_91_t['heating_kWh/m2'] - dem75_91_t['hot_water_kWh/m2'] + dem75_91_t['cooling_kWh/m2']
dem75_91_t = pd.DataFrame([dem75_91_t['total']])
dem75_91_t = pd.concat([dem75_91_t]*len(df75_91_t), ignore_index=True)

df75_91_t_t = pd.concat([df75_91_t[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']].T, dem75_91_t.T]).T

df75_91_t_t.update(df75_91_t_t.iloc[:, 7:8759+8].mul(df75_91_t_t.oppervlakt, 0))
df75_91_t_t = df75_91_t_t.iloc[:, 0:8759+8]

In [28]:
dem75_91_sd = pd.read_excel('data/Residential_demand/Semi-detached/corner_1974_1992/LT_corner_1975_1995_cavity_fin.xlsx')

df75_91_sd = df75_91_sd[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']]
df75_91_sd = df75_91_sd.reset_index(drop=True)

dem75_91_sd['total'] = - dem75_91_sd['heating_kWh/m2'] - dem75_91_sd['hot_water_kWh/m2'] + dem75_91_sd['cooling_kWh/m2']
dem75_91_sd = pd.DataFrame([dem75_91_sd['total']])
dem75_91_sd = pd.concat([dem75_91_sd]*len(df75_91_sd), ignore_index=True)

df75_91_sd_t = pd.concat([df75_91_sd[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']].T, dem75_91_sd.T]).T

df75_91_sd_t.update(df75_91_sd_t.iloc[:, 7:8759+8].mul(df75_91_sd_t.oppervlakt, 0))
df75_91_sd_t = df75_91_sd_t.iloc[:, 0:8759+8]

In [29]:
dem92_a = pd.read_excel('data/Residential_demand/Apartment/4_apartment_1992/LT_appart_1992_cavityw.xlsx')

df92_a = df92_a[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']]
df92_a = df92_a.reset_index(drop=True)

dem92_a['total'] = - dem92_a['heating_kWh/m2'] - dem92_a['hot_water_kWh/m2'] + dem92_a['cooling_kWh/m2']
dem92_a = pd.DataFrame([dem92_a['total']])
dem92_a = pd.concat([dem92_a]*len(df92_a), ignore_index=True)

df92_a_t = pd.concat([df92_a[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']].T, dem92_a.T]).T

df92_a_t.update(df92_a_t.iloc[:, 7:8759+8].mul(df92_a_t.oppervlakt, 0))

In [30]:
dem92_t = pd.read_excel('data/Residential_demand/Terraced/row_1992/LT_row_1995_fin.xlsx')

df92_t = df92_t[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']]
df92_t = df92_t.reset_index(drop=True)

dem92_t['total'] = - dem92_t['heating_kWh/m2'] - dem92_t['hot_water_kWh/m2'] + dem92_t['cooling_kWh/m2']
dem92_t = pd.DataFrame([dem92_t['total']])
dem92_t = pd.concat([dem92_t]*len(df92_t), ignore_index=True)

df92_t_t = pd.concat([df92_t[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']].T, dem92_t.T]).T

df92_t_t.update(df92_t_t.iloc[:, 7:8759+8].mul(df92_t_t.oppervlakt, 0))
df92_t_t = df92_t_t.iloc[:, 0:8759+8]

In [31]:
dem92_sd = pd.read_excel('data/Residential_demand/Semi-detached/corner_1992/LT_corner_1992.xlsx')

df92_sd = df92_sd[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']]
df92_sd = df92_sd.reset_index(drop=True)

dem92_sd['total'] = - dem92_sd['heating_kWh/m2'] - dem92_sd['hot_water_kWh/m2'] + dem92_sd['cooling_kWh/m2']
dem92_sd = pd.DataFrame([dem92_sd['total']])
dem92_sd = pd.concat([dem92_sd]*len(df92_sd), ignore_index=True)

df92_sd_t = pd.concat([df92_sd[['point','oppervlakt', 'openbare_r','geometry','postcode', 'x', 'y']].T, dem92_sd.T]).T

df92_sd_t.update(df92_sd_t.iloc[:, 7:8759+8].mul(df92_sd_t.oppervlakt, 0))
df92_sd_t = df92_sd_t.iloc[:, 0:8759+8]

In [32]:
frames = [df45_p_a_t, df45_p_t_t, df45_p_sd_t, df45_np_a_t, df45_np_t_t, df45_np_sd_t, df46_74_a_t, df46_74_t_t, df75_91_a_t, df75_91_t_t, df75_91_sd_t, df92_a_t, df92_t_t, df92_sd_t] 
#df46_74_sd_t has been left out, since it is not represented in the data

dfdem = pd.concat(frames)
dfdem.head()

,point,oppervlakt,openbare_r,geometry,postcode,x,y,0,1,2,...,8750,8751,8752,8753,8754,8755,8756,8757,8758,8759
0,"121231.0, 488105.0",95,Herenmarkt,POINT (121231 488105),1013ED,121231.0,488105.0,-1.09421,-1.06153,-1.082905,...,-1.30017,-1.30796,-1.20897,-1.14266,-1.050795,-0.95285,-0.900885,-0.902025,-1.01308,-1.005005
1,"121231.0, 488105.0",57,Herenmarkt,POINT (121231 488105),1013ED,121231.0,488105.0,-0.656526,-0.636918,-0.649743,...,-0.780102,-0.784776,-0.725382,-0.685596,-0.630477,-0.57171,-0.540531,-0.541215,-0.607848,-0.603003
2,"121231.0, 488105.0",114,Herenmarkt,POINT (121231 488105),1013ED,121231.0,488105.0,-1.313052,-1.273836,-1.299486,...,-1.560204,-1.569552,-1.450764,-1.371192,-1.260954,-1.14342,-1.081062,-1.08243,-1.215696,-1.206006
3,"121245.0, 487930.0",110,Herengracht,POINT (121245 487930),1015BB,121245.0,487930.0,-1.26698,-1.22914,-1.25389,...,-1.50546,-1.51448,-1.39986,-1.32308,-1.21671,-1.1033,-1.04313,-1.04445,-1.17304,-1.16369
4,"121245.0, 487930.0",87,Herengracht,POINT (121245 487930),1015BB,121245.0,487930.0,-1.002066,-0.972138,-0.991713,...,-1.190682,-1.197816,-1.107162,-1.046436,-0.962307,-0.87261,-0.825021,-0.826065,-0.927768,-0.920373


dfdem is the dataframe containing the demand for each building for each hour of the year

### 3. Add the excess waste heat to the dataframe

In [33]:
# Import the waste heat data
wh = gpd.read_file('data/Waste_heat_map_HC/arch_non_residential_WH.gpkg')
wh.columns

Index(['BAG_pand_i', 'BAG_pand_b', 'Q_c_kWh', 'Elec_kWh', 'Pec_4_cool', 'COP',
       'Q_v_kWh', 'Delta_T', 'H', 'Volume_m3', 'ACH_hr', 'Q_all_kWh',
       'Q_all_GJ', 'BAG_pand_o', 'Meest_voor', 'Hoofdfun_1', 'Gebruiksop',
       'LNG', 'LAT', 'identifica', 'rdf_seeals', 'bouwjaar', 'status',
       'gebruiksdo', 'oppervlakt', 'oppervla_1', 'aantal_ver', 'fuuid',
       'woningtype', 'wwr', 'geometry'],
      dtype='object')

In [34]:
# Set the data set to the right CRS
wh = wh.to_crs(28992)
wh['x'] = wh['geometry'].x
wh['y'] = wh['geometry'].y

wh['point'] = wh['x'].astype(str) + ', ' + wh['y'].astype(str)

In [35]:
bag_r = bag.drop_duplicates(subset=['pandidenti'])

In [36]:
# Prepare data for merge and get rid of redundant information
wh.rename(columns = {'identifica':'pandidenti'}, inplace = True)
wh = wh[['pandidenti', 'Q_c_kWh', 'geometry', 'Hoofdfun_1','oppervlakt', 'point', 'x', 'y']]
wh['pandidenti'] = wh['pandidenti'].astype(str)

In [37]:
# Merge the waste heat data with the building data to get more information of the building
df_wh = wh.merge(arch, how='inner', on='pandidenti')
df_wh.head()

,pandidenti,Q_c_kWh,geometry,Hoofdfun_1,oppervlakt,point,x,y,woningtype
0,0363100012177046,3.265724e+03,POINT (120857.003 488655.081),K,53.0,"120857.00322794568, 488655.08084049827",120857.003228,488655.080840,None
1,0363100012176707,3.881898e+03,POINT (120898.061 488616.075),K,63.0,"120898.06063839531, 488616.07539828605",120898.060638,488616.075398,None
2,0363100012167506,2.091716e+03,POINT (121247.000 486785.092),D,20.0,"121247.00030534054, 486785.0919997972",121247.000305,486785.092000,tussenwoning/geschakeld
3,0363100012176704,4.852372e+03,POINT (120883.050 488631.089),K,63.0,"120883.04968252344, 488631.08917179465",120883.049683,488631.089172,None
4,0363100012176709,3.881898e+03,POINT (120909.028 488607.098),K,63.0,"120909.02830964497, 488607.0981126377",120909.028310,488607.098113,None
...,...,...,...,...,...,...,...,...,...
2916,0363100012169158,1.708970e+05,POINT (121499.011 486575.081),H,37.0,"121499.01143952922, 486575.0813093517",121499.011440,486575.081309,appartement
2917,0363100012170951,2.079445e+05,POINT (121579.043 487822.077),H,58.0,"121579.04302546664, 487822.077481819",121579.043025,487822.077482,appartement
2918,0363100012185779,7.430161e+05,POINT (121135.027 486626.078),H,1.0,"121135.02661978942, 486626.07789459947",121135.026620,486626.077895,None
2919,0363100012165684,1.004310e+07,POINT (121888.985 485777.107),K,1.0,"121888.98536194908, 485777.10716088535",121888.985362,485777.107161,appartement


In [38]:
# Merge this dataframe with the adress data, to get the ZIP code etc.
df_wh = df_wh.merge(bag_r, how='inner', on='pandidenti')

In [39]:
df_wh.columns

Index(['pandidenti', 'Q_c_kWh', 'geometry_x', 'Hoofdfun_1', 'oppervlakt_x',
       'point', 'x', 'y', 'woningtype', 'gid', 'identifica', 'oppervlakt_y',
       'status', 'gebruiksdo', 'openbare_r', 'huisnummer', 'huisletter',
       'toevoeging', 'postcode', 'woonplaats', 'bouwjaar', 'pandstatus',
       'rdf_seeals', 'Objectnumm', 'Orde', 'SELECTIE_O', 'Gebied',
       'SELECTIE_G', 'Architectn', 'Architect', 'Bouwjaar_2', 'Stadsdeel',
       'Brug', '_errors', 'layer', 'path', 'date_f', 'Joined lay',
       'geometry_y'],
      dtype='object')

In [41]:
wh = df_wh[['Q_c_kWh','geometry_x','postcode','pandidenti','openbare_r', 'Hoofdfun_1','oppervlakt_x', 'point', 'x', 'y']]
wh.rename(columns = {'geometry_x':'geometry', 'oppervlakt_x':'oppervlakt'}, inplace = True)
wh.head()

C:\Users\sarah\AppData\Local\Temp\ipykernel_11888\790724981.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wh.rename(columns = {'geometry_x':'geometry', 'oppervlakt_x':'oppervlakt'}, inplace = True)


,Q_c_kWh,geometry,postcode,pandidenti,openbare_r,Hoofdfun_1,oppervlakt,point,x,y
0,3.265724e+03,POINT (120857.003 488655.081),1013JB,0363100012177046,Tussen de Bogen,K,53.0,"120857.00322794568, 488655.08084049827",120857.003228,488655.080840
1,3.881898e+03,POINT (120898.061 488616.075),1013JB,0363100012176707,Tussen de Bogen,K,63.0,"120898.06063839531, 488616.07539828605",120898.060638,488616.075398
2,2.091716e+03,POINT (121247.000 486785.092),1012XH,0363100012167506,Kalverstraat,D,20.0,"121247.00030534054, 486785.0919997972",121247.000305,486785.092000
3,4.852372e+03,POINT (120883.050 488631.089),1013JB,0363100012176704,Tussen de Bogen,K,63.0,"120883.04968252344, 488631.08917179465",120883.049683,488631.089172
4,3.881898e+03,POINT (120909.028 488607.098),1013JB,0363100012176709,Tussen de Bogen,K,63.0,"120909.02830964497, 488607.0981126377",120909.028310,488607.098113
...,...,...,...,...,...,...,...,...,...,...
2875,1.708970e+05,POINT (121499.011 486575.081),1017CV,0363100012169158,Rembrandtplein,H,37.0,"121499.01143952922, 486575.0813093517",121499.011440,486575.081309
2876,2.079445e+05,POINT (121579.043 487822.077),1012NB,0363100012170951,Nieuwezijds Armsteeg,H,58.0,"121579.04302546664, 487822.077481819",121579.043025,487822.077482
2877,7.430161e+05,POINT (121135.027 486626.078),1017BM,0363100012185779,Reguliersdwarsstraat,H,1.0,"121135.02661978942, 486626.07789459947",121135.026620,486626.077895
2878,1.004310e+07,POINT (121888.985 485777.107),1017WT,0363100012165684,Oosteinde,K,1.0,"121888.98536194908, 485777.10716088535",121888.985362,485777.107161


In [42]:
# Look up the uniqie main functions, this is needed to couple the waste heat data to
wh.Hoofdfun_1.unique()

array(['K', 'D', 'O', 'U', 'H', 'L'], dtype=object)

In [43]:
wh = wh[['point', 'oppervlakt', 'Hoofdfun_1', 'Q_c_kWh', 'openbare_r','postcode','geometry', 'x', 'y']]

The waste heat data will be converted from yearly data to hourly data. The datasets of Nina Voulis are used to make an hourly distribution of the waste heat. This will be calculated per function.

In [44]:
# Data of offices
wh_K = wh.loc[wh['Hoofdfun_1'] == 'K']
whe_K = pd.read_csv('data/ElectricityDemand/MediumOffice_ElectricityDemandProfile_kWhPerh_2014_NL.csv')

whe_K['wh_kWh_per_h'] = 0
whe_K['wh_kWh_per_h'] = whe_K['ElectricityTotal_kWh_per_h'].div(whe_K.ElectricityTotal_kWh_per_h.sum())

wh_K = wh_K[['point','oppervlakt', 'Hoofdfun_1', 'Q_c_kWh', 'openbare_r','geometry','postcode', 'x', 'y']]
wh_K = wh_K.reset_index(drop=True)

whe_K = pd.DataFrame([whe_K['wh_kWh_per_h']])
whe_K = pd.concat([whe_K]*len(wh_K), ignore_index=True)

wh_K_t = pd.concat([wh_K[['point','oppervlakt', 'Hoofdfun_1', 'Q_c_kWh', 'openbare_r','geometry','postcode', 'x', 'y']].T, whe_K.T]).T

wh_K_t.update(wh_K_t.iloc[:, 9:8759+10].mul(wh_K_t.Q_c_kWh, 0))
wh_K_t.drop(columns=[8760], inplace=True)

In [45]:
# Data of retail
wh_D = wh.loc[wh['Hoofdfun_1'] == 'D']
whe_D = pd.read_csv('data/ElectricityDemand/StandAloneRetail_ElectricityDemandProfile_kWhPerh_2014_NL.csv')

whe_D['wh_kWh_per_h'] = 0
whe_D['wh_kWh_per_h'] = whe_D['ElectricityTotal_kWh_per_h'].div(whe_D.ElectricityTotal_kWh_per_h.sum())

wh_D = wh_D[['point','oppervlakt', 'Hoofdfun_1', 'Q_c_kWh', 'openbare_r','geometry','postcode', 'x', 'y']]
wh_D = wh_D.reset_index(drop=True)

whe_D = pd.DataFrame([whe_D['wh_kWh_per_h']])
whe_D = pd.concat([whe_D]*len(wh_D), ignore_index=True)

wh_D_t = pd.concat([wh_D[['point','oppervlakt', 'Hoofdfun_1', 'Q_c_kWh', 'openbare_r','geometry','postcode', 'x', 'y']].T, whe_D.T]).T

wh_D_t.update(wh_D_t.iloc[:, 9:8759+10].mul(wh_D_t.Q_c_kWh, 0))
wh_D_t.drop(columns=[8760], inplace=True)

In [46]:
# Data of schools
wh_O = wh.loc[wh['Hoofdfun_1'] == 'O']
whe_O = pd.read_csv('data/ElectricityDemand/SecondarySchool_ElectricityDemandProfile_kWhPerh_2014_NL.csv')

whe_O['wh_kWh_per_h'] = 0
whe_O['wh_kWh_per_h'] = whe_O['ElectricityTotal_kWh_per_h'].div(whe_O.ElectricityTotal_kWh_per_h.sum())

wh_O = wh_O[['point','oppervlakt', 'Hoofdfun_1', 'Q_c_kWh', 'openbare_r','geometry','postcode', 'x', 'y']]
wh_O = wh_O.reset_index(drop=True)

whe_O = pd.DataFrame([whe_O['wh_kWh_per_h']])
whe_O = pd.concat([whe_O]*len(wh_O), ignore_index=True)

wh_O_t = pd.concat([wh_O[['point','oppervlakt', 'Hoofdfun_1', 'Q_c_kWh', 'openbare_r','geometry','postcode', 'x', 'y']].T, whe_O.T]).T

wh_O_t.update(wh_O_t.iloc[:, 9:8759+10].mul(wh_O_t.Q_c_kWh, 0))
wh_O_t.drop(columns=[8760], inplace=True)

In [47]:
# Data of supermarkets
wh_U = wh.loc[wh['Hoofdfun_1'] == 'U']
whe_U = pd.read_csv('data/ElectricityDemand/Supermarket_ElectricityDemandProfile_kWhPerh_2014_NL.csv')

whe_U['wh_kWh_per_h'] = 0
whe_U['wh_kWh_per_h'] = whe_U['ElectricityTotal_kWh_per_h'].div(whe_U.ElectricityTotal_kWh_per_h.sum())

wh_U = wh_U[['point','oppervlakt', 'Hoofdfun_1', 'Q_c_kWh', 'openbare_r','geometry','postcode', 'x', 'y']]
wh_U = wh_U.reset_index(drop=True)

whe_U = pd.DataFrame([whe_U['wh_kWh_per_h']])
whe_U = pd.concat([whe_U]*len(wh_U), ignore_index=True)

wh_U_t = pd.concat([wh_U[['point','oppervlakt', 'Hoofdfun_1', 'Q_c_kWh', 'openbare_r','geometry','postcode', 'x', 'y']].T, whe_U.T]).T

wh_U_t.update(wh_U_t.iloc[:, 9:8759+10].mul(wh_U_t.Q_c_kWh, 0))
wh_U_t.drop(columns=[8760], inplace=True)

In [48]:
# Data of restaurants
wh_H = wh.loc[wh['Hoofdfun_1'] == 'H']
whe_H = pd.read_csv('data/ElectricityDemand/Restaurant_ElectricityDemandProfile_kWhPerh_2014_NL.csv')

whe_H['wh_kWh_per_h'] = 0
whe_H['wh_kWh_per_h'] = whe_H['ElectricityTotal_kWh_per_h'].div(whe_H.ElectricityTotal_kWh_per_h.sum())

wh_H = wh_H[['point','oppervlakt', 'Hoofdfun_1', 'Q_c_kWh', 'openbare_r','geometry','postcode', 'x', 'y']]
wh_H = wh_H.reset_index(drop=True)

whe_H = pd.DataFrame([whe_H['wh_kWh_per_h']])
whe_H = pd.concat([whe_H]*len(wh_H), ignore_index=True)

wh_H_t = pd.concat([wh_H[['point','oppervlakt', 'Hoofdfun_1', 'Q_c_kWh', 'openbare_r','geometry','postcode', 'x', 'y']].T, whe_H.T]).T

wh_H_t.update(wh_H_t.iloc[:, 9:8759+10].mul(wh_H_t.Q_c_kWh, 0))
wh_H_t.drop(columns=[8760], inplace=True)

In [49]:
# Data of hotels
wh_L = wh.loc[wh['Hoofdfun_1'] == 'L']
whe_L = pd.read_csv('data/ElectricityDemand/LargeHotel_ElectricityDemandProfile_kWhPerh_2014_NL.csv')

whe_L['wh_kWh_per_h'] = 0
whe_L['wh_kWh_per_h'] = whe_L['ElectricityTotal_kWh_per_h'].div(whe_L.ElectricityTotal_kWh_per_h.sum())

wh_L = wh_L[['point','oppervlakt', 'Hoofdfun_1', 'Q_c_kWh', 'openbare_r','geometry','postcode', 'x', 'y']]
wh_L = wh_L.reset_index(drop=True)

whe_L = pd.DataFrame([whe_L['wh_kWh_per_h']])
whe_L = pd.concat([whe_L]*len(wh_L), ignore_index=True)

wh_L_t = pd.concat([wh_L[['point','oppervlakt', 'Hoofdfun_1', 'Q_c_kWh', 'openbare_r','geometry','postcode', 'x', 'y']].T, whe_L.T]).T

wh_L_t.update(wh_L_t.iloc[:, 9:8759+10].mul(wh_L_t.Q_c_kWh, 0))
wh_L_t.drop(columns=[8760], inplace=True)

In [50]:
frames = [wh_K_t, wh_D_t, wh_O_t, wh_U_t, wh_H_t, wh_L_t] 

whdem = pd.concat(frames)

whdem.head()

,point,oppervlakt,Hoofdfun_1,Q_c_kWh,openbare_r,geometry,postcode,x,y,0,...,8750,8751,8752,8753,8754,8755,8756,8757,8758,8759
0,"120857.00322794568, 488655.08084049827",53.0,K,3265.723564,Tussen de Bogen,POINT (120857.00322794568 488655.08084049827),1013JB,120857.003228,488655.08084,0.299048,...,0.65194,0.668398,0.725616,0.706723,0.573756,0.566407,0.526413,0.534754,0.187646,0.175558
1,"120898.06063839531, 488616.07539828605",63.0,K,3881.897822,Tussen de Bogen,POINT (120898.06063839531 488616.07539828605),1013JB,120898.060638,488616.075398,0.355472,...,0.774948,0.794511,0.862524,0.840067,0.682012,0.673276,0.625736,0.635651,0.223051,0.208682
2,"120883.04968252344, 488631.08917179465",63.0,K,4852.372277,Tussen de Bogen,POINT (120883.04968252344 488631.08917179465),1013JB,120883.049683,488631.089172,0.44434,...,0.968685,0.993138,1.078155,1.050084,0.852515,0.841595,0.78217,0.794564,0.278814,0.260853
3,"120909.02830964497, 488607.0981126377",63.0,K,3881.897822,Tussen de Bogen,POINT (120909.02830964497 488607.0981126377),1013JB,120909.02831,488607.098113,0.355472,...,0.774948,0.794511,0.862524,0.840067,0.682012,0.673276,0.625736,0.635651,0.223051,0.208682
4,"121512.96596134552, 485911.05056289234",21.0,K,6700.89505,Fokke Simonszstraat,POINT (121512.96596134552 485911.05056289234),1017TK,121512.965961,485911.050563,0.613612,...,1.337708,1.371477,1.488881,1.450117,1.177283,1.162203,1.08014,1.097255,0.385029,0.360225


### 4. Join the two dataframes into one dataframe

In [54]:
df_total = pd.concat([whdem, dfdem])

df_total.head()

,point,oppervlakt,Hoofdfun_1,Q_c_kWh,openbare_r,geometry,postcode,x,y,0,...,8750,8751,8752,8753,8754,8755,8756,8757,8758,8759
0,"120857.00322794568, 488655.08084049827",53.0,K,3265.723564,Tussen de Bogen,POINT (120857.00322794568 488655.08084049827),1013JB,120857.003228,488655.08084,0.299048,...,0.65194,0.668398,0.725616,0.706723,0.573756,0.566407,0.526413,0.534754,0.187646,0.175558
1,"120898.06063839531, 488616.07539828605",63.0,K,3881.897822,Tussen de Bogen,POINT (120898.06063839531 488616.07539828605),1013JB,120898.060638,488616.075398,0.355472,...,0.774948,0.794511,0.862524,0.840067,0.682012,0.673276,0.625736,0.635651,0.223051,0.208682
2,"120883.04968252344, 488631.08917179465",63.0,K,4852.372277,Tussen de Bogen,POINT (120883.04968252344 488631.08917179465),1013JB,120883.049683,488631.089172,0.44434,...,0.968685,0.993138,1.078155,1.050084,0.852515,0.841595,0.78217,0.794564,0.278814,0.260853
3,"120909.02830964497, 488607.0981126377",63.0,K,3881.897822,Tussen de Bogen,POINT (120909.02830964497 488607.0981126377),1013JB,120909.02831,488607.098113,0.355472,...,0.774948,0.794511,0.862524,0.840067,0.682012,0.673276,0.625736,0.635651,0.223051,0.208682
4,"121512.96596134552, 485911.05056289234",21.0,K,6700.89505,Fokke Simonszstraat,POINT (121512.96596134552 485911.05056289234),1017TK,121512.965961,485911.050563,0.613612,...,1.337708,1.371477,1.488881,1.450117,1.177283,1.162203,1.08014,1.097255,0.385029,0.360225


In [55]:
df_total.to_csv('data/residential_and_wh_hourly_innercity.csv')

In [57]:
# Check which columns are needed to sum for a timeframe
df_total.iloc[:, 0+9:8759+10].head()

,0,1,2,3,4,5,6,7,8,9,...,8750,8751,8752,8753,8754,8755,8756,8757,8758,8759
0,0.299048,0.280187,0.323928,0.285571,0.325398,0.269248,0.302114,0.282473,0.321069,0.240685,...,0.65194,0.668398,0.725616,0.706723,0.573756,0.566407,0.526413,0.534754,0.187646,0.175558
1,0.355472,0.333052,0.385046,0.339452,0.386793,0.32005,0.359116,0.33577,0.381648,0.286097,...,0.774948,0.794511,0.862524,0.840067,0.682012,0.673276,0.625736,0.635651,0.223051,0.208682
2,0.44434,0.416316,0.481308,0.424315,0.483492,0.400062,0.448895,0.419712,0.47706,0.357621,...,0.968685,0.993138,1.078155,1.050084,0.852515,0.841595,0.78217,0.794564,0.278814,0.260853
3,0.355472,0.333052,0.385046,0.339452,0.386793,0.32005,0.359116,0.33577,0.381648,0.286097,...,0.774948,0.794511,0.862524,0.840067,0.682012,0.673276,0.625736,0.635651,0.223051,0.208682
4,0.613612,0.574912,0.664663,0.585958,0.667679,0.552467,0.619903,0.579603,0.658797,0.493857,...,1.337708,1.371477,1.488881,1.450117,1.177283,1.162203,1.08014,1.097255,0.385029,0.360225


In [58]:
df_total['coldest_day'] = df_total.iloc[:, 1056+9:1079+10].sum(axis=1)
df_total['coldest_month'] = df_total.iloc[:, 0+9:743+10].sum(axis=1)
df_total['whole_year'] = df_total.iloc[:, 0+9:8759+10].sum(axis=1)
df_total2 = df_total[['point', 'oppervlakt', 'Hoofdfun_1', 'Q_c_kWh', 'openbare_r', 'geometry', 'postcode', \
                      'x', 'y', 'coldest_day', 'coldest_month', 'whole_year']]

In [59]:
df_total2.head()

,point,oppervlakt,Hoofdfun_1,Q_c_kWh,openbare_r,geometry,postcode,x,y,coldest_day,coldest_month,whole_year
0,"120857.00322794568, 488655.08084049827",53.0,K,3265.723564,Tussen de Bogen,POINT (120857.00322794568 488655.08084049827),1013JB,120857.003228,488655.08084,12.483029,338.288855,3265.723564
1,"120898.06063839531, 488616.07539828605",63.0,K,3881.897822,Tussen de Bogen,POINT (120898.06063839531 488616.07539828605),1013JB,120898.060638,488616.075398,14.838318,402.116941,3881.897822
2,"120883.04968252344, 488631.08917179465",63.0,K,4852.372277,Tussen de Bogen,POINT (120883.04968252344 488631.08917179465),1013JB,120883.049683,488631.089172,18.547897,502.646177,4852.372277
3,"120909.02830964497, 488607.0981126377",63.0,K,3881.897822,Tussen de Bogen,POINT (120909.02830964497 488607.0981126377),1013JB,120909.02831,488607.098113,14.838318,402.116941,3881.897822
4,"121512.96596134552, 485911.05056289234",21.0,K,6700.89505,Fokke Simonszstraat,POINT (121512.96596134552 485911.05056289234),1017TK,121512.965961,485911.050563,25.613763,694.130435,6700.895050


In [60]:
df_total2.to_csv('data/residential_and_wh_total_innercity.csv')

In [61]:
dfdem.shape

(54573, 8767)

In [62]:
whdem.shape

(2880, 8769)

In [63]:
df_total.shape

(57453, 8772)

In [64]:
wh.shape

(2880, 9)